## Julia Basics

In [ ]:
a = 2         # Integer
b = 2.        # double precision floating point
b = 2.0f0     # single precision floating point
c = 1//3      # rational number
d = 1 + 3im   # complex number
println(a, ", ", b, ", ", c, ", ", d)

In [ ]:
v = [1, 5, 7]

In [ ]:
isa(v, Vector)

Vectors are different from matrices and different from scalars

In [ ]:
b = randn(3)  # vector of normal random numbers
A = [1 2 3;
     2 4 5;
     3 5 6]

In [ ]:
isa(A, Matrix)

In [ ]:
size(A)

In [ ]:
res = b.'*A*b   # this is NOT dotproduct

In [ ]:
typeof(res)     # Vector

In [ ]:
res = dot(b, A*b)

## Familiar syntax to Matlab

In [ ]:
det(A)                   # determinant
vals, vectors = eig(A)   # eigenvalues
A^2                      # matrix to power
A.^4                     # matrix to power elementwise

In [ ]:
u,s,v = svd(A)           # singular value decomposition
s

In [ ]:
norm(A)                  # matrix norm
A\b                      # solve linear system Ax = b

In [ ]:
?norm

# matrix manipulations

In [ ]:
A[:,[1,2]]

In [ ]:
zeros(3,3)

In [ ]:
A + 3I + 3eye(3) + 3speye(3)

In [ ]:
diagm([1, 4, 5])

In [ ]:
vcat(A, [5 5 5])

In [ ]:
hcat(A, [5 5 5].')

Matrices have column major ordering

In [ ]:
a = 1:4

In [ ]:
b = collect(a)

In [ ]:
c = reshape(b,2,2)

In [ ]:
c[1,2] == c[3] == 3

Julia specializes functions depending on input types through multiple dispatch 

In [ ]:
d = diagm([2,3,4])

In [ ]:
isa(d, Matrix)

In [ ]:
A + d

In [ ]:
D = Diagonal([2,3,4])

Diagonal is a different type!

In [ ]:
isa(D, Matrix)            

In [ ]:
isa(D, AbstractMatrix)

In [ ]:
subtypes(AbstractMatrix{Int64})

But it can still be used as if it were a normal dense matrix

In [ ]:
A + D

In [ ]:
methods(+);

In [ ]:
@which A + D

## Loops

In [ ]:
A = randn(3,3)

for loops as we are used to them

In [ ]:
counts = 0
for row = 1:size(A,1)
  for col = 1:size(A,2)
    if A[row,col] ≤ 0
      counts += 1
    else
      nothing
    end
  end
end
counts

In [ ]:
for row = 1:size(A,1), col = 1:size(A,2)
  if A[row,col] ≤ 0
    counts += 1
  else
    nothing
  end
end
counts

better looking, and more efficient way


In [ ]:
for idx in eachindex(A)
  counts += A[idx] ≤ 0 ? 1 : 0
end
counts

can also iterate only over elements

In [ ]:
for elem in A
  counts += elem ≤ 0 ? 1 : 0
end
counts

works for any "iterable" type

In [ ]:
for elem in A
  counts += elem ≤ 0
end
println(counts)

isa(true, Integer)

In [ ]:
filter(x->x ≤ zero(x), A)

In [ ]:
counts = filter(x->x ≤ zero(x), A) |> length     # chaining the output of filter to length

## while loop

In [ ]:
kmax = 6
k = 1

while k < kmax
  # do something
  # do not forget to increment k
  # string interpolation with "$"
  println("k: $k")
  k += 1
end

# Functions, just in time compilation and multiple dispatch

In [1]:
f(x,y) = x + y

f (generic function with 1 method)

In [2]:
f(1,1)

2

In [3]:
f(x::Int, y::Int) = 2x+y

f (generic function with 2 methods)

In [4]:
f(1,1)  # dispatches to most specialized method

3

In [6]:
@time f(1,1)
@time f(1,1)
@time f(1,1.0)
@time f(1,1.0)

  0.000003 seconds (4 allocations: 160 bytes)
  0.000002 seconds (4 allocations: 160 bytes)
  0.000003 seconds (5 allocations: 176 bytes)
  0.000002 seconds (5 allocations: 176 bytes)


2.0

Julia, like many languages, compiles to LLVM and we can inspect the generated code (LLVM is a type of portable assembly language).

In [7]:
@code_llvm f(1,1)


define i64 @julia_f_70671(i64, i64) #0 {
top:
  %2 = shl i64 %0, 1
  %3 = add i64 %2, %1
  ret i64 %3
}


In [8]:
a = 2.
b = √a       # \sqrt TAB a

1.4142135623730951

In [9]:
a = -1.
b = √a

LoadError: DomainError:
sqrt will only return a complex result if called with a complex argument. Try sqrt(complex(x)).

In [10]:
a = complex(a)
b = sqrt(a)

0.0 + 1.0im

In [11]:
function mysqrt(x::Real)
  try
    return √x
  catch err
    return √Complex128(x)
  finally
    println("mysqrt: I have finished")
  end
end

mysqrt (generic function with 1 method)

In [12]:
mysqrt(-1)

mysqrt: I have finished


0.0 + 1.0im

## Type stability
Type stability is the idea that there is only 1 possible type which can be outputted from a method.
For example, the reasonable type to output from +(::Int,::Int) is an Int.
Julia is fast because it can specialize via multiple dispatch to type stable methods.

In [18]:
@code_warntype sqrt(1.)

Variables:
  #self#::Base.#sqrt
  x::Float64

Body:
  begin 
      return (Base.Math.box)(Base.Math.Float64,(Base.Math.sqrt_llvm)(x::Float64))::Float64
  end::Float64


In [19]:
@code_warntype mysqrt(1.)

Variables:
  #self#::#mysqrt
  x::Float64
  #temp#@_3::Any
  #temp#@_4::Union{}
  #temp#@_5::Bool
  #temp#@_6::Any
  err::Any
  #temp#@_8::Union{}

Body:
  begin 
      NewvarNode(:(#temp#@_4::Union{}))
      NewvarNode(:(#temp#@_6::Any))
      #temp#@_5::Bool = false
      $(Expr(:enter, 17))
      $(Expr(:enter, 10)) # line 3:
      #temp#@_3::Any = (Base.Math.box)(Base.Math.Float64,(Base.Math.sqrt_llvm)(x::Float64))::Float64
      $(Expr(:leave, 2))
      goto 20
      10: 
      $(Expr(:leave, 1))
      err::Any = $(Expr(:the_exception)) # line 5:
      #temp#@_3::Any = $(Expr(:invoke, LambdaInfo for sqrt(::Complex{Float64}), :(Main.√), :($(Expr(:new, Complex{Float64}, :(x), :((Base.box)(Float64,(Base.sitofp)(Float64,0))))))))
      $(Expr(:leave, 1))
      goto 20
      17: 
      $(Expr(:leave, 1))
      #temp#@_5::Bool = true
      20: 
      #temp#@_6::Any = $(Expr(:the_exception)) # line 7:
      # meta: location coreio.jl println 5
      SSAValue(0) = (Core.typeassert)(Base.S

# sets and dictionaries and tuples

In [20]:
a = (1,2.0,3) |> typeof

Tuple{Int64,Float64,Int64}

In [21]:
b = [1,2,3] |> typeof

Array{Int64,1}

Tuples cannot be changed

In [22]:
a[1] = 2

LoadError: MethodError: no method matching setindex!(::Type{Tuple{Int64,Float64,Int64}}, ::Int64, ::Int64)

In [23]:
mycol = Set{Int}()
push!(mycol, (1,2,3,1,2,3)...)
push!(mycol, 1, 2, 3, 4, 1, 2)

Set([4,2,3,1])

In [24]:
col1 = mycol
col2 = copy(mycol)   # pass by reference is default so we need to copy 

Set([4,2,3,1])

In [25]:
pop!(col2, 1)        # remove 1 from the set

1

In [26]:
res = col1 ∪ col2    # union() \cup

Set([4,2,3,1])

In [27]:
res = col1 ∩ col2    # intersect() \cap

Set([4,2,3])

In [28]:
res = setdiff(col1,col2)

Set([1])

In [29]:
1 ∈ res              # in()

true

In [30]:
res ∋ 1

true

In [31]:
1 ∉ res

false

In [32]:
dict = Dict("a" => 1, "b" => 2, "c" => 3)

Dict{String,Int64} with 3 entries:
  "c" => 3
  "b" => 2
  "a" => 1

In [33]:
dict["b"]

2

In [34]:
for (key, value) in dict
    println(key, " ==> ", value)
end

c ==> 3
b ==> 2
a ==> 1


# Custom types

In [35]:
type MyType
  α::Float64
end

In [36]:
m = MyType(5)

MyType(5.0)

In [37]:
m.α = 3.

3.0

In [38]:
m

MyType(3.0)

In [39]:
fieldnames(m)

1-element Array{Symbol,1}:
 :α

In [40]:
# Some not-trivial constructor
function (::Type{MyType})(x::Vector)
  MyType(sum(x))
end

In [46]:
m2 = MyType([1,2,3])

MyType(6.0)

In [41]:
# Function call-overload: we are making MyType a callable object (i.e., a functor)
function (m::MyType)(x)
  return m.α * x
end

In [42]:
m(5)

15.0

In [43]:
import Base.+
+(m1::MyType, m2::MyType) = MyType(m1.α+m2.α)

+ (generic function with 164 methods)

In [47]:
mres = m+m2

MyType(9.0)

# Function of functions
Function is a type and can be assigned to variables

In [48]:
g = (m,x)->m(x)

(::#1) (generic function with 1 method)

In [49]:
g(sqrt, 2)

1.4142135623730951

In [50]:
isa(g, Function)

true

Our custom type is also callable

In [51]:
g(m, 3)

9.0

In [52]:
g(m, randn(6))

6-element Array{Float64,1}:
 -0.142929
 -0.203639
 -2.05155 
  1.09605 
 -4.15817 
  1.74397 